In [1]:
import librosa
import pandas as pd
import numpy as np
import csv
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
#from sklearn.metrics import confusion_matrix
#import tensorflow as tf
#from tensorflow import keras



#--------------------------------Debut Declaration des fonctions--------------------------------------
"""
    Nom fonction: spectralIrregularityValue
    description : Elle permet de calculer le spectral irregularity d'un signal
    argument : signal -->  transformation de serie de fourier
"""
def spectralIrregularityValue(signal):
    _sum = 0
    k = 1
    while k <len(signal)-1:
        _sum += np.abs(20*np.log(signal[k]) - (20*np.log(signal[k-1])+20*np.log(signal[k])+20*np.log(signal[k+1]))/3)
        k+=1
    return _sum
"""
    Nom fonction: peakValue
    description : Elle permet de calculer le peak d'un signal
"""
def peakValue(signal,n):
    max_peak = np.abs(signal[0])
    i=1
    while i<n :
        if(max_peak < np.abs(signal[i])):
            max_peak = np.abs(signal[i])
        i+=1
    return max_peak
#Function that Calculate Root Mean Square
"""
    Nom fonction: rmsValue
    description : Elle permet de Calculer le Root Mean Square
"""
def rmsValue(signal,n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calculate Square
    for i in range(n):
        square+=signal[i]**2
    #Calculate Mean
    mean = (square / (float)(n))
    #Calculate Root
    root = np.sqrt(mean)
    return root
#--------------------------------Fin Declaration des fonctions--------------------------------------

#--------------------------------MAIN--------------------------------------

# etape 1: Creation du dataset  a partir du repertoire instruments 
header = 'filename zero_crossing spectral_irregularity crest_factor'
header += ' label' #label  va correspondre au nom de l'instrument
header = header.split()

file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
musics = 'blues country hiphop jazz pop reggae'.split()
#parcours du dossier insctrument et de ses sous repertoire
for g in musics:
    for filename in os.listdir(f'./music/{g}'):
        songname = f'./music/{g}/{filename}'
        #chargement de l'audio
        y, sr = librosa.load(songname)
        #Extraction des features
        zero_crossings_array = librosa.zero_crossings(y)
        zero_crossing =sum(zero_crossings_array)
        #audio fast fourrier transformation
        audio_ft = np.fft.fft(y)
        spectral_irregularity = spectralIrregularityValue(audio_ft)
        rms = rmsValue(y,len(y))
        peak = peakValue(y,len(y))
        crest_factor = peak/rms
        to_append = f'{filename} {np.mean(zero_crossing)} {spectral_irregularity} {crest_factor}' 
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

#    etape 2: lecture du dataset a partir  du fichier csv
data = pd.read_csv('data.csv')
print(data)
# Suppression des colonnes non neccessaire (filename)
data = data.drop(['filename'],axis=1)
data.head()
# recuperation de la liste des instrument et le stocker dans la variable instrument_list
music_list = data.iloc[:, -1]
#affecter un numero a un instrument ie piano=1,violen=2
encoder = LabelEncoder()
y = encoder.fit_transform(music_list)
print(y)

             filename  zero_crossing  spectral_irregularity  crest_factor  \
0     blues.00002.wav        50564.0           1.616541e+07      4.628418   
1     blues.00004.wav        67226.0           1.778807e+07      8.316786   
2     blues.00005.wav        62327.0           1.805464e+07      8.248955   
3     blues.00003.wav        22078.0           1.901394e+07      5.220570   
4     blues.00001.wav        37140.0           1.600716e+07      6.353945   
5     blues.00000.wav        55032.0           1.742477e+07      6.293168   
6   country.00004.wav        82229.0           1.783582e+07      3.481964   
7   country.00005.wav        91071.0           1.811433e+07      3.294301   
8   country.00003.wav        67921.0           1.741171e+07      3.913811   
9   country.00000.wav        71218.0           1.812084e+07      3.096103   
10  country.00001.wav        84183.0           1.786754e+07      3.117361   
11  country.00002.wav        49746.0           1.699963e+07      4.090650   